<a href="https://colab.research.google.com/github/Gideon1712/SimpleTrading/blob/main/SimpleTrading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

App.py

In [ ]:
pip install Flask

In [11]:
from flask import Flask, render_template, request, redirect, url_for, flash, session
from datetime import datetime, timedelta
import sqlite3
import time

app = Flask(__name__)
app.secret_key = 'supersecretkey'

# Database setup
def init_db():
    conn = sqlite3.connect('users.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS users
                 (id INTEGER PRIMARY KEY, username TEXT UNIQUE, password TEXT, failed_attempts INTEGER, lock_until INTEGER)''')
    conn.commit()
    conn.close()

init_db()

# Routes
@app.route('/')
def home():
    return render_template('login.html')

@app.route('/login', methods=['POST'])
def login():
    username = request.form['username']
    password = request.form['password']

    conn = sqlite3.connect('users.db')
    c = conn.cursor()
    c.execute('SELECT * FROM users WHERE username=?', (username,))
    user = c.fetchone()

    if user:
        user_id, db_username, db_password, failed_attempts, lock_until = user

        if lock_until and time.time() < lock_until:
            wait_time = int((lock_until - time.time()) / 60)
            flash(f'Account locked. Try again in {wait_time} minutes.')
            return redirect(url_for('home'))

        if password == db_password:
            session['user'] = username
            flash('Welcome User')
            c.execute('UPDATE users SET failed_attempts=0, lock_until=NULL WHERE username=?', (username,))
            conn.commit()
            return redirect(url_for('home'))
        else:
            failed_attempts += 1
            lock_until = None
            if failed_attempts > 5:
                lock_until = time.time() + (5 * 60 * (failed_attempts // 5))
            c.execute('UPDATE users SET failed_attempts=?, lock_until=? WHERE username=?', (failed_attempts, lock_until, username))
            conn.commit()
            flash('Incorrect username or password.')
    else:
        flash('User does not exist.')

    conn.close()
    return redirect(url_for('home'))

@app.route('/signup', methods=['GET', 'POST'])
def signup():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        conn = sqlite3.connect('users.db')
        c = conn.cursor()
        try:
            c.execute('INSERT INTO users (username, password, failed_attempts) VALUES (?, ?, 0)', (username, password))
            conn.commit()
            flash('User created successfully.')
            return redirect(url_for('home'))
        except sqlite3.IntegrityError:
            flash('Username already exists.')
        conn.close()
    return render_template('signup.html')

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
